# 2. Raster data preparations

In this exercise we prepare the raster data for the other excercises. We are going to use command line commands, **not python**. These commands could be run in a terminal window as well.

in Jupyter Notebooks, command line commands start with **!** or **%**
* **%** means the command will be ran so that the result persists for other code cells as well. You can navigate folders
* **!** runs the command in a separate subprocess. This means that switching folders with `cd` would not work

The raw data to be prepared is:
* Forest stands from Forest center (Metsäkeskus). https://www.metsaan.fi/paikkatietoaineistot
* Sentinel 2A satellite image (10m x 10m) from ESA. https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-2/data-products
* a False color aerial image (0.5m x 0.5m) from the NLS https://www.maanmittauslaitos.fi/kartat-ja-paikkatieto/asiantuntevalle-kayttajalle/tuotekuvaukset/ilmakuva
* Building footprints from the Topographic Database (NLS) https://www.maanmittauslaitos.fi/kartat-ja-paikkatieto/asiantuntevalle-kayttajalle/tuotekuvaukset/maastotietokanta-0

Buildings have been separated from the data with the following command

`ogr2ogr -f "GPKG" -spat 422000 6768000 428000 6762000 buildings_M4311G.gpkg /appl/data/geo/mml/maastotietokanta/2019/gpkg/MTK-rakennus_19-01-23.gpkg -sql "SELECT * FROM rakennus"`

## 2.1 Download the raster data from Allas 

We use `wget` to download the raster data involved in this course. 

In [ ]:
%env NOTEBOOK_HOME=/home/jovyan/work/geocomputing/machineLearning

In [ ]:
! wget https://a3s.fi/gis-courses/gis_ml/raster_data.zip -O $NOTEBOOK_HOME/data/raster_data.zip
! unzip -q -o $NOTEBOOK_HOME/data/raster_data.zip -d $NOTEBOOK_HOME/data

## 2.2 Sentinel satellite preparations

Create the false color composite as a virtual raster (.vrt) from the different bands

* **B08** = infrared
* **B04** = red
* **B03** = green

In [ ]:
%cd /home/jovyan/work/geocomputing/machineLearning/data/forest/S2B_MSIL2A_20180829T100019_N0208_R122_T34VFM_20180829T184909.SAFE/GRANULE/L2A_T34VFM_A007727_20180829T100017/IMG_DATA/R10m
! gdalbuildvrt -separate T34VFM_20180829T100019.vrt T34VFM_20180829T100019_B08_10m.jp2 T34VFM_20180829T100019_B04_10m.jp2 T34VFM_20180829T100019_B03_10m.jp2

### 2.2.1 Scaling and clipping

Scale the pixel values from 0 to 10 000 -> 0 to 1. In Sentinel images, the original values have been multiplied by 10 000 to get rid of decimals (0.0001 takes more disk space than 10 000)

In [ ]:
# Clip the training area
! gdal_translate -projwin 614500 6668500 644500 6640500 T34VFM_20180829T100019.vrt $NOTEBOOK_HOME/data/forest/T34VFM_20180829T100019_clipped_scaled.tif -ot Float32 -scale 0 10000 0 1
# Clip the original image also a bit smaller for predictions
! gdal_translate -projwin 604500 6698500 677000 6640000 T34VFM_20180829T100019.vrt $NOTEBOOK_HOME/data/forest/T34VFM_20180829T100019_scaled.tif -ot Float32 -scale 0 10000 0 1

## 2.3 Forest stand preparations

### 2.3.1 Merging the forest data

Merge the two GeoPackage files to file called forest_clipped.gpkg. Exclude polygons, that have no main tree species value

Parameter explanation
* **-t_srs** new coordinate system, epsg:3067 is the code for ETRS-TM35FIN
* **-spat** bbox of study area in UTM 34N coordinates
* **-spat_srs** EPSG code of the spat coodrinates - UTM 34N
* **-where** select only polygons that have the maintreespecies value. In the dataset were some polygons with -1 value.

This will take a few minutes

In [ ]:
%cd /home/jovyan/work/geocomputing/machineLearning/data/forest/
! ogr2ogr -f GPKG -t_srs epsg:32634 -spat_srs epsg:32634 -where "maintreespecies>0" forest_clipped.gpkg MV_Salo.gpkg stand
! ogr2ogr -f GPKG -t_srs epsg:32634 -spat_srs epsg:32634 -where "maintreespecies>0" -append -update forest_clipped.gpkg MV_Uusimaa.gpkg stand

### 2.3.2 Rasterizing, classifying and clipping

We need to rasterize the **.gpkg** into a **.tif** file and at the same time discard unrelevant tree species from the dataset

In [ ]:
# Rasterize forest stand polygons and clip to the same extent as the full Sentinel image
! gdal_rasterize -a maintreespecies -ot Byte -tr 10 10 -te 604500 6640000 677000 6698500 forest_clipped.gpkg -l stand forest_species.tif

# Classify the forest main tree species to three classes: Pine (1), Spruce (2), Deciduous trees (3), No trees (0)
! gdal_calc.py -A forest_species.tif --outfile=forest_species_reclassified.tif --calc="3*(A>=3)+1*(A==1)+2*(A==2)" --NoDataValue=0 --quiet

# Some excercises use only the spruce data. Rasterize it from the original gpkg file and scale it from 0 2 to 0 1
! gdal_rasterize -a maintreespecies -ot Byte -tr 10 10 -te 604500 6640000 677000 6698500 -where "maintreespecies=2" forest_clipped.gpkg -l stand forest_spruce02.tif
! gdal_translate forest_spruce02.tif -ot Byte -scale 0 2 0 1 forest_spruce.tif

# Clip training area for both 3-class and 1-class datasets and for spruce dataset, change the value of spruce from 2 to 1
! gdal_translate forest_spruce.tif forest_spruce_clip.tif -ot Byte -projwin 614500 6668500 644500 6640500
! gdal_translate forest_species_reclassified.tif forest_species_reclassified_clip.tif -ot Byte -projwin 614500 6668500 644500 6640500

## 2.4 Aerial image preparations

We are using an aerial image from NLS to train with building footprints

The aerial image has so many pixels it is necessary to clip it for it to be usable in this course

In [ ]:
# Go back to the "buildings" data folder
%cd /home/jovyan/work/geocomputing/machineLearning/data/buildings/
! gdal_translate -projwin 422000 6765000 425000 6762000 M4311G.jp2 M4311G_clip.tif

## 2.5 Building footprint preparations

Rasterize the building footprint polygons and clip them to the same extent as the aerial image

In [ ]:
! gdal_rasterize buildings_M4311G.gpkg -ot Byte -te 422000 6762000 425000 6765000 -tr 0.5 0.5 -burn 1 -l SELECT building_footprints_clip.tif

## 2.6 Plotting the datasets

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import rasterio
import numpy as np
from rasterio.plot import show

%cd /home/jovyan/work/geocomputing/machineLearning

In [ ]:
### Function to normalize band values and enhance contrast. Just like what QGIS does automatically
def normalize(array):
    min_percent = 2   # Low percentile
    max_percent = 98  # High percentile
    lo, hi = np.percentile(array, (min_percent, max_percent))
    return (array - lo) / (hi - lo)

### 2.6.1 The Sentinel image and forest stands

In [ ]:
### This is the clipped sentinel image used in the training phases 
sentinel = rasterio.open("data/forest/T34VFM_20180829T100019_clipped_scaled.tif")

### Read the bands separately and apply the normalize function to each of them to increase contrast
nir, red, green = sentinel.read(1), sentinel.read(2), sentinel.read(3)
nirn, redn, greenn = normalize(nir), normalize(red), normalize(green)
stacked = np.dstack((nirn, redn, greenn))

### Create a subplot for two images and plot the sentinel image 

fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(20, 10))
ax1.imshow(stacked)

### This is the forest species file
species = rasterio.open("data/forest/forest_species_reclassified_clip.tif")

### We plot it a bit differently as it is not an RGB image
show(species, ax=ax2, cmap='hsv')

### 2.6.2 Aerial image and building footprints

In [ ]:
aerial = rasterio.open("data/buildings/M4311G_clip.tif")

### Read the bands separately and apply the normalize function to each of them to increase contrast
nir, red, green = aerial.read(1), aerial.read(2), aerial.read(3)
nirn, redn, greenn = normalize(nir), normalize(red), normalize(green)
stacked = np.dstack((nirn, redn, greenn))

### Create a subplot for two images and plot the sentinel image 

fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(20, 10))
ax1.imshow(stacked)

### This is the forest species file
#buildings = rasterio.open("data/buildings/building_footprints_clip.tif")

### We plot it a bit differently as it is not an RGB image
#show(buildings, ax=ax2, cmap='hsv')